### 0. Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir data
!mkdir data/baseline_catboost

!wget https://storage.yandexcloud.net/datasouls-ods/materials/0433a4ca/transactions.zip -P data
!wget https://storage.yandexcloud.net/datasouls-ods/materials/0554f0cf/clickstream.zip -P data
!wget https://storage.yandexcloud.net/datasouls-ods/materials/acfacf11/train_matching.csv -P data
!wget https://storage.yandexcloud.net/datasouls-ods/materials/b949c04c/mcc_codes.csv -P data
!wget https://storage.yandexcloud.net/datasouls-ods/materials/705abbab/click_categories.csv -P data
!wget https://storage.yandexcloud.net/datasouls-ods/materials/e33f2201/currency_rk.csv -P data
!wget https://storage.yandexcloud.net/datasouls-ods/materials/b99fed70/puzzle.csv -P data
!wget https://storage.yandexcloud.net/datasouls-ods/materials/f76e8087/sample_submission.csv -P data
!wget https://storage.yandexcloud.net/datasouls-ods/materials/24687252/baseline_catboost.zip -P data
    
!unzip data/transactions.zip -d data
!unzip data/clickstream.zip -d data
!unzip data/baseline_catboost.zip -d data/baseline_catboost

!rm data/transactions.zip
!rm data/clickstream.zip
!rm data/baseline_catboost.zip

Mounted at /content/drive
--2022-05-11 15:13:35--  https://storage.yandexcloud.net/datasouls-ods/materials/0433a4ca/transactions.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262900158 (251M) [application/zip]
Saving to: ‘data/transactions.zip’

transactions.zip    100%[===================>] 250.72M  21.0MB/s    in 13s     

2022-05-11 15:13:50 (19.1 MB/s) - ‘data/transactions.zip’ saved [262900158/262900158]

--2022-05-11 15:13:50--  https://storage.yandexcloud.net/datasouls-ods/materials/0554f0cf/clickstream.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 492816210 (470M) [applicat

In [ ]:
!pip install catboost
!pip install shap

     |████████████████████████████████| 76.6 MB 53 kB/s 
     |████████████████████████████████| 564 kB 5.3 MB/s 


In [ ]:
import numpy as np 
import pandas as pd 
import sys
import pickle
from catboost import CatBoostClassifier
import catboost
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from catboost import Pool
import shap
import multiprocessing
from psutil import virtual_memory
from tqdm import tqdm_notebook
from catboost import CatBoostRanker

from sklearn.metrics import plot_roc_curve, roc_auc_score


data, output_path = sys.argv[1:]
input_folder = 'data'
ram_gb = round(virtual_memory().total / 1024**3, 1)

print(data)
print(output_path)
print(catboost.__version__, np.__version__, pd.__version__)
print('CPU:', multiprocessing.cpu_count())
print('RAM GB:', ram_gb)

-f
/root/.local/share/jupyter/runtime/kernel-cdeb7b31-d8fd-4624-9d1d-6a3455b5e113.json
1.0.5 1.21.6 1.3.5
CPU: 40
RAM GB: 35.2


### 1. Read and preprocess data

### 1.1 Clickstream

In [ ]:
all_dicts = {} 

clickstream = pd.read_csv('/content/data/clickstream.csv')
clickstream['timestamp'] = pd.to_datetime(clickstream['timestamp'])

# чтобы уменьшить объём файла закодируем user_id числами
all_dicts['rtk_le'] = LabelEncoder().fit(clickstream['user_id'])
clickstream['user_id'] = all_dicts['rtk_le'].transform(clickstream['user_id'])+1
clickstream_dtypes = {'user_id':np.int16, 'cat_id':np.int16, 'new_uid':np.int32}
clickstream = clickstream.astype(clickstream_dtypes)
clickstream['hour'] = clickstream['timestamp'].dt.hour # // 3 

clickstream.head()

,user_id,cat_id,timestamp,new_uid,hour
0,1,165,2021-01-30 20:08:12,1873448,20
1,1,165,2021-01-31 20:06:29,1873448,20
2,1,308,2021-01-31 20:12:00,1873448,20
3,1,931,2021-01-31 22:12:00,1873448,22
4,1,931,2021-02-01 16:57:00,1873448,16


### 1.2 Transactions

In [ ]:
transactions = pd.read_csv('/content/data/transactions.csv')
transactions['transaction_dttm'] = pd.to_datetime(transactions['transaction_dttm'])
transactions['hour'] = transactions['transaction_dttm'].dt.hour 

# Аналогично, чтобы уменьшить объём файла закодируем user_id числами
all_dicts['bank_le'] = LabelEncoder().fit(transactions['user_id'])
transactions['user_id'] = all_dicts['bank_le'].transform(transactions['user_id'])+1
transactions_dtypes = {'user_id':np.int16, 'mcc_code':np.int16, 'currency_rk':np.int8}
transactions = transactions.astype(transactions_dtypes)
transactions.head()

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,hour
0,1,5411,48,-361.07230,2020-08-03 08:05:23,8
1,1,5499,48,-137.31398,2020-08-05 01:27:40,1
2,1,5499,48,-138.84981,2020-08-05 03:28:11,3
3,1,4829,48,-309.47653,2020-08-06 00:36:29,0
4,1,5411,48,-133.47370,2020-08-09 00:30:13,0


### Dates embeddings

In [ ]:
clickstream['weekofyear'] = clickstream['timestamp'].dt.week.astype('int16') # номер недели в году
clickstream['day_of_week'] = clickstream['timestamp'].dt.dayofweek.astype('int16') # номер дня недели
clickstream['month'] = clickstream['timestamp'].dt.month.astype('int16') # номер месяца в году
clickstream['day'] = clickstream['timestamp'].dt.day.astype('int16') # номер дня в месяце
clickstream['hour'] = clickstream['timestamp'].dt.hour.astype('int16') # час от 0 до 24

clickstream['cl_is_afternoon'] = (clickstream['hour'] > 12).astype('int16') # признак второй половины дня
clickstream['cl_early_morning'] = ((clickstream['hour'] > 4) & (clickstream['hour'] <= 8)).astype('int16') # признак "раннее утро"
# clickstream['cl_morning'] = ((clickstream['hour'] > 8) & (clickstream['hour'] <= 12)).astype('int16') # признак "утро"
# clickstream['cl_noon'] = ((clickstream['hour'] > 12) & (clickstream['hour'] <= 17)).astype('int16') # признак "день"
# clickstream['cl_eve'] = ((clickstream['hour'] > 18) & (clickstream['hour'] <= 21)).astype('int16') # признак "вечер"
# clickstream['cl_night'] = ((clickstream['hour'] > 21) & (clickstream['hour'] <= 24)).astype('int16') # признак "ночь"
# clickstream['cl_late_night'] = (clickstream['hour'] <= 4).astype('int16') # признак "глубокая ночь"

clickstream['cl_is_weekend'] = (clickstream['day_of_week'] > 4).astype('int16') # признак "выходной день"
# clickstream['cl_is_month_start'] = clickstream['timestamp'].dt.is_month_start.astype('int16') # признак "начало месяца"
# clickstream['cl_is_month_end'] = clickstream['timestamp'].dt.is_month_end.astype('int16') # признак "конец месяца"

# clickstream['timestamp'] = clickstream['timestamp'].dt.strftime("%Y-%m-%d") # преобразование даты в формат "yy-MM-dd"

# Дни недели
# clickstream['cl_Monday'] = (clickstream['day_of_week'] == 0).astype('int16')
# clickstream['cl_Tuesday'] = (clickstream['day_of_week'] == 1).astype('int16')
# clickstream['cl_Wednesday'] = (clickstream['day_of_week'] == 2).astype('int16')
# clickstream['cl_Thursday'] = (clickstream['day_of_week'] == 3).astype('int16')
# clickstream['cl_Friday'] = (clickstream['day_of_week'] == 4).astype('int16')
# clickstream['cl_Saturday'] = (clickstream['day_of_week'] == 5).astype('int16')
# clickstream['cl_Sunday'] = (clickstream['day_of_week'] == 6).astype('int16')

cols = [
 'cl_is_afternoon',
 'cl_early_morning',
 'cl_morning',
 'cl_noon',
 'cl_eve',
 'cl_night',
 'cl_late_night',
 'cl_is_weekend',
 'cl_is_month_start',
 'cl_is_month_end',
 'cl_Monday',
 'cl_Tuesday',
 'cl_Wednesday',
 'cl_Thursday',
 'cl_Friday',
 'cl_Saturday',
 'cl_Sunday'
 ]

cols = ['cl_is_afternoon', 'cl_early_morning', 'cl_is_weekend']

new_clickstream = pd.pivot_table(clickstream.drop(['cat_id', 'new_uid', 'timestamp', 'date', 'weekday', 'weekofyear', 'day_of_week', 'month', 'day'], axis=1).fillna(0), index='user_id', values = cols, aggfunc = np.count_nonzero).fillna(0)
new_clickstream.head()


,cl_early_morning,cl_is_afternoon,cl_is_weekend
user_id,,,
1,600,11091,4384
2,839,3966,2087
3,936,4199,1492
4,1439,2392,1646
5,1142,2538,1807


In [ ]:
clickstream.head()

,user_id,cat_id,timestamp,new_uid,hour,date,weekday,weekofyear,day_of_week,month,day,cl_is_afternoon,cl_early_morning,cl_is_weekend
0,1,165,2021-01-30 20:08:12,1873448,20,2021-01-30,5,4,5,1,30,1,0,1
1,1,165,2021-01-31 20:06:29,1873448,20,2021-01-31,6,4,6,1,31,1,0,1
2,1,308,2021-01-31 20:12:00,1873448,20,2021-01-31,6,4,6,1,31,1,0,1
3,1,931,2021-01-31 22:12:00,1873448,22,2021-01-31,6,4,6,1,31,1,0,1
4,1,931,2021-02-01 16:57:00,1873448,16,2021-02-01,0,5,0,2,1,1,0,0


In [ ]:
transactions['weekofyear'] = transactions['transaction_dttm'].dt.week.astype('int16') # номер недели в году
transactions['day_of_week'] = transactions['transaction_dttm'].dt.dayofweek.astype('int16') # номер дня недели
transactions['month'] = transactions['transaction_dttm'].dt.month.astype('int16') # номер месяца в году
transactions['day'] = transactions['transaction_dttm'].dt.day.astype('int16') # номер дня в месяце
transactions['hour'] = transactions['transaction_dttm'].dt.hour.astype('int16') # час от 0 до 24

transactions['tr_is_afternoon'] = (transactions['hour'] > 12).astype('int16') # признак второй половины дня
transactions['tr_early_morning'] = ((transactions['hour'] > 4) & (transactions['hour'] <= 8)).astype('int16') # признак "раннее утро"
# transactions['tr_morning'] = ((transactions['hour'] > 8) & (transactions['hour'] <= 12)).astype('int16') # признак "утро"
# transactions['tr_noon'] = ((transactions['hour'] > 12) & (transactions['hour'] <= 17)).astype('int16') # признак "день"
# transactions['tr_eve'] = ((transactions['hour'] > 18) & (transactions['hour'] <= 21)).astype('int16') # признак "вечер"
# transactions['tr_night'] = ((transactions['hour'] > 21) & (transactions['hour'] <= 24)).astype('int16') # признак "ночь"
# transactions['tr_late_night'] = (transactions['hour'] <= 4).astype('int16') # признак "глубокая ночь"

transactions['tr_is_weekend'] = (transactions['day_of_week'] > 4).astype('int16') # признак "выходной день"
# transactions['tr_is_month_start'] = transactions['transaction_dttm'].dt.is_month_start.astype('int16') # признак "начало месяца"
# transactions['tr_is_month_end'] = transactions['transaction_dttm'].dt.is_month_end.astype('int16') # признак "конец месяца"

# transactions['transaction_dttm'] = transactions['transaction_dttm'].dt.strftime("%Y-%m-%d") # преобразование даты в формат "yy-MM-dd"

transactions['operation_type'] = np.where(transactions['transaction_amt'] > 0, 1, 0).astype('int16') # признак "тип операции" (поступение - 1, списание - 0)

transactions['transaction_amt_abs'] = transactions['transaction_amt'].abs().astype('float32') # модуль суммы транзакции

# Дни недели
# transactions['tr_Monday'] = (transactions['day_of_week'] == 0).astype('int16')
# transactions['tr_Tuesday'] = (transactions['day_of_week'] == 1).astype('int16')
# transactions['tr_Wednesday'] = (transactions['day_of_week'] == 2).astype('int16')
# transactions['tr_Thursday'] = (transactions['day_of_week'] == 3).astype('int16')
# transactions['tr_Friday'] = (transactions['day_of_week'] == 4).astype('int16')
# transactions['tr_Saturday'] = (transactions['day_of_week'] == 5).astype('int16')
# transactions['tr_Sunday'] = (transactions['day_of_week'] == 6).astype('int16')

transactions.head()

cols = [
 'tr_is_afternoon',
 'tr_early_morning',
 'tr_morning',
 'tr_noon',
 'tr_eve',
 'tr_night',
 'tr_late_night',
 'tr_is_weekend',
 'tr_is_month_start',
 'tr_is_month_end', 
 'tr_Monday',
 'tr_Tuesday',
 'tr_Wednesday',
 'tr_Thursday',
 'tr_Friday',
 'tr_Saturday',
 'tr_Sunday']

cols = ['tr_is_afternoon', 'tr_early_morning', 'tr_is_weekend', 'operation_type']

new_transactions = pd.pivot_table(transactions.drop(['mcc_code', 'currency_rk', 'transaction_amt', 'transaction_dttm', 
                                                     'transaction_amt_abs'], axis=1).fillna(0), index='user_id', values = cols, aggfunc = np.count_nonzero).fillna(0)

new_transactions.head()

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,operation_type,tr_early_morning,tr_is_afternoon,tr_is_weekend
user_id,,,,
1,47,137,275,215
2,3,127,292,242
3,40,314,159,362
4,50,28,667,225
5,13,304,265,301


### 1.3 Other files

In [ ]:
puzzle = pd.read_csv('./data/puzzle.csv')
print(puzzle.shape)
puzzle.head(2)

puzzle['bank'] = all_dicts['bank_le'].transform(puzzle['bank'])+1
puzzle['rtk'] = all_dicts['rtk_le'].transform(puzzle['rtk'])+1

train = pd.read_csv('./data/train_matching.csv')
print(train.shape)
train.head(2)

train['bank'] = all_dicts['bank_le'].transform(train['bank'])+1
train.loc[train.rtk=='0', 'rtk'] = 0
train.loc[train.rtk!=0, 'rtk'] = all_dicts['rtk_le'].transform(train.loc[train.rtk!=0, 'rtk'])+1

(4952, 2)
(17581, 2)


In [ ]:
puzzle.head()

,bank,rtk
0,9720,6550
1,3027,15496
2,20887,957
3,1517,4508
4,12223,14723


In [ ]:
train.head()

,bank,rtk
0,2091,17319
1,6347,9604
2,21724,13623
3,6456,7502
4,4866,14760


### 2. Делаем небейзлайновые эмбеддинги

#### Эмбеддинги на основе часов проведённой транзакции и часов в кликстриме

Транзакции:

In [ ]:
tr_sv = pd.pivot_table(transactions, index='user_id', columns='hour', values = 'transaction_amt', aggfunc = 'count').fillna(0)

In [ ]:
tr_sv.head() # к-во транзакицй в каждый час

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
user_id,,,,,,,,,,,,,,,,,,,,,
1,83.0,90.0,80.0,93.0,73.0,63.0,38.0,20.0,16.0,7.0,...,1.0,1.0,0.0,0.0,0.0,36.0,51.0,57.0,60.0,69.0
2,4.0,12.0,25.0,29.0,31.0,32.0,34.0,26.0,35.0,46.0,...,77.0,61.0,53.0,22.0,8.0,5.0,0.0,0.0,0.0,1.0
3,9.0,16.0,26.0,49.0,72.0,83.0,85.0,83.0,63.0,84.0,...,51.0,19.0,9.0,5.0,4.0,4.0,1.0,2.0,0.0,2.0
4,2.0,1.0,1.0,4.0,3.0,10.0,9.0,4.0,5.0,3.0,...,12.0,11.0,15.0,10.0,6.0,74.0,225.0,226.0,77.0,0.0
5,39.0,48.0,38.0,26.0,21.0,36.0,67.0,86.0,115.0,85.0,...,59.0,26.0,14.0,7.0,20.0,23.0,23.0,3.0,0.0,5.0


In [ ]:
tr_sv['summs'] = tr_sv.sum(axis=1)
for i in tr_sv.columns[:-1]:
    tr_sv[i] /= tr_sv['summs']
tr_sv.columns = ['trans_h_'+ str(i) for i in tr_sv.columns]

In [ ]:
tr_sv.head() # какая часть всех транзакций проводилась в каждый отельный час

,trans_h_0,trans_h_1,trans_h_2,trans_h_3,trans_h_4,trans_h_5,trans_h_6,trans_h_7,trans_h_8,trans_h_9,...,trans_h_15,trans_h_16,trans_h_17,trans_h_18,trans_h_19,trans_h_20,trans_h_21,trans_h_22,trans_h_23,trans_h_summs
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.098225,0.106509,0.094675,0.110059,0.086391,0.074556,0.044970,0.023669,0.018935,0.008284,...,0.001183,0.000000,0.000000,0.000000,0.042604,0.060355,0.067456,0.071006,0.081657,845.0
2,0.005348,0.016043,0.033422,0.038770,0.041444,0.042781,0.045455,0.034759,0.046791,0.061497,...,0.081551,0.070856,0.029412,0.010695,0.006684,0.000000,0.000000,0.000000,0.001337,748.0
3,0.009288,0.016512,0.026832,0.050568,0.074303,0.085655,0.087719,0.085655,0.065015,0.086687,...,0.019608,0.009288,0.005160,0.004128,0.004128,0.001032,0.002064,0.000000,0.002064,969.0
4,0.002766,0.001383,0.001383,0.005533,0.004149,0.013831,0.012448,0.005533,0.006916,0.004149,...,0.015214,0.020747,0.013831,0.008299,0.102351,0.311203,0.312586,0.106501,0.000000,723.0
5,0.037901,0.046647,0.036929,0.025267,0.020408,0.034985,0.065112,0.083576,0.111759,0.082604,...,0.025267,0.013605,0.006803,0.019436,0.022352,0.022352,0.002915,0.000000,0.004859,1029.0


Кликстрим:

In [ ]:
cl_sv = pd.pivot_table(clickstream, index='user_id', columns='hour', values = 'timestamp', aggfunc = 'count').fillna(0)

In [ ]:
cl_sv.head() # количество кликов каждый отдельный час

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
user_id,,,,,,,,,,,,,,,,,,,,,
1,14.0,10.0,74.0,150.0,168.0,134.0,91.0,140.0,235.0,223.0,...,481.0,594.0,734.0,1163.0,1866.0,2177.0,1941.0,1158.0,431.0,79.0
2,26.0,30.0,63.0,102.0,124.0,194.0,227.0,216.0,202.0,152.0,...,324.0,427.0,550.0,553.0,545.0,490.0,424.0,264.0,106.0,40.0
3,33.0,28.0,64.0,115.0,119.0,138.0,207.0,265.0,326.0,356.0,...,418.0,473.0,477.0,471.0,461.0,488.0,426.0,331.0,188.0,105.0
4,126.0,112.0,118.0,189.0,228.0,209.0,246.0,429.0,555.0,506.0,...,478.0,380.0,288.0,196.0,123.0,63.0,62.0,74.0,83.0,127.0
5,53.0,148.0,289.0,335.0,298.0,277.0,269.0,285.0,311.0,272.0,...,420.0,518.0,496.0,428.0,194.0,64.0,22.0,4.0,7.0,5.0


In [ ]:
cl_sv['summs'] = cl_sv.sum(axis=1)
for i in cl_sv.columns[:-1]:
    cl_sv[i] /= cl_sv['summs']
cl_sv.columns = ['click_h_'+ str(i) for i in cl_sv.columns]

In [ ]:
cl_sv.head() # сколько всего кликов и какая часть кликов была сделана в каждый отдельный час

,click_h_0,click_h_1,click_h_2,click_h_3,click_h_4,click_h_5,click_h_6,click_h_7,click_h_8,click_h_9,...,click_h_15,click_h_16,click_h_17,click_h_18,click_h_19,click_h_20,click_h_21,click_h_22,click_h_23,click_h_summs
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.001061,0.000758,0.005607,0.011365,0.012729,0.010153,0.006895,0.010608,0.017806,0.016896,...,0.045007,0.055614,0.088119,0.141385,0.164949,0.147068,0.087741,0.032656,0.005986,13198.0
2,0.004559,0.005260,0.011047,0.017885,0.021743,0.034017,0.039804,0.037875,0.035420,0.026653,...,0.074873,0.096440,0.096967,0.095564,0.085920,0.074347,0.046291,0.018587,0.007014,5703.0
3,0.004788,0.004063,0.009286,0.016686,0.017266,0.020023,0.030035,0.038450,0.047301,0.051654,...,0.068630,0.069211,0.068340,0.066889,0.070807,0.061811,0.048027,0.027278,0.015235,6892.0
4,0.019833,0.017629,0.018574,0.029750,0.035889,0.032898,0.038722,0.067527,0.087360,0.079647,...,0.059814,0.045333,0.030852,0.019361,0.009917,0.009759,0.011648,0.013065,0.019991,6353.0
5,0.008656,0.024171,0.047199,0.054712,0.048669,0.045239,0.043933,0.046546,0.050792,0.044423,...,0.084599,0.081006,0.069900,0.031684,0.010452,0.003593,0.000653,0.001143,0.000817,6123.0


#### Дополнительные:

1. Transactions

In [ ]:
bankclient_embed2 = transactions.pivot_table(index = 'user_id', 
                            values=['transaction_amt'],
                            columns=['currency_rk'],
                            aggfunc=['sum', 'mean', 'count']).fillna(0)
bankclient_embed2.columns = [f'{str(i[0])}-{str(i[2])}' for i in bankclient_embed2.columns]
bankclient_embed2.head()

,sum--1,sum-48,sum-50,sum-60,mean--1,mean-48,mean-50,mean-60,count--1,count-48,count-50,count-60
user_id,,,,,,,,,,,,
1,0.000000,-1.834270e+05,0.0,0.0,0.000000,-217.073402,0.0,0.0,0.0,845.0,0.0,0.0
2,0.750996,-7.176788e+05,0.0,0.0,0.750996,-960.748099,0.0,0.0,1.0,747.0,0.0,0.0
3,0.000000,8.016948e+05,0.0,0.0,0.000000,827.342388,0.0,0.0,0.0,969.0,0.0,0.0
4,-0.378836,1.261829e+06,0.0,0.0,-0.378836,1747.685734,0.0,0.0,1.0,722.0,0.0,0.0
5,0.837259,-4.172432e+05,0.0,0.0,0.837259,-405.878595,0.0,0.0,1.0,1028.0,0.0,0.0


In [ ]:
transactions['hour'] = transactions['transaction_dttm'].dt.hour.astype('category')
transactions['weekday'] = transactions['transaction_dttm'].dt.dayofweek.astype('category')
bankclient_embed4 = transactions.pivot_table(index = 'user_id', 
                            values=['transaction_dttm'],
                            columns=['weekday','hour'],
                            aggfunc=['count']).fillna(0)
bankclient_embed4.columns = [f'bnk-{str(i[0])}-weekday-{str(i[2])}-nhour-{str(i[3])}' for i in bankclient_embed4.columns]
bankclient_embed4['summs'] = bankclient_embed4.sum(axis=1)
for i in bankclient_embed4.columns[:-1]:
    bankclient_embed4[i] /= bankclient_embed4['summs']
bankclient_embed4.columns = [str(i) for i in bankclient_embed4.columns]
bankclient_embed4.head()

,bnk-count-weekday-0-nhour-0,bnk-count-weekday-0-nhour-1,bnk-count-weekday-0-nhour-2,bnk-count-weekday-0-nhour-3,bnk-count-weekday-0-nhour-4,bnk-count-weekday-0-nhour-5,bnk-count-weekday-0-nhour-6,bnk-count-weekday-0-nhour-7,bnk-count-weekday-0-nhour-8,bnk-count-weekday-0-nhour-9,...,bnk-count-weekday-6-nhour-15,bnk-count-weekday-6-nhour-16,bnk-count-weekday-6-nhour-17,bnk-count-weekday-6-nhour-18,bnk-count-weekday-6-nhour-19,bnk-count-weekday-6-nhour-20,bnk-count-weekday-6-nhour-21,bnk-count-weekday-6-nhour-22,bnk-count-weekday-6-nhour-23,summs
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.013018,0.009467,0.010651,0.015385,0.010651,0.015385,0.004734,0.004734,0.001183,0.001183,...,0.000000,0.000000,0.000000,0.000000,0.008284,0.009467,0.007101,0.004734,0.004734,845
2,0.000000,0.004011,0.010695,0.006684,0.004011,0.005348,0.009358,0.009358,0.006684,0.009358,...,0.018717,0.012032,0.001337,0.001337,0.001337,0.000000,0.000000,0.000000,0.000000,748
3,0.001032,0.004128,0.001032,0.006192,0.005160,0.007224,0.011352,0.007224,0.005160,0.013416,...,0.002064,0.000000,0.001032,0.000000,0.000000,0.000000,0.000000,0.000000,0.001032,969
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.002766,0.002766,0.001383,0.001383,0.000000,...,0.001383,0.002766,0.000000,0.001383,0.006916,0.044260,0.049793,0.009682,0.000000,723
5,0.001944,0.007775,0.005831,0.001944,0.003887,0.001944,0.013605,0.011662,0.018465,0.012634,...,0.001944,0.000972,0.001944,0.005831,0.001944,0.002915,0.000972,0.000000,0.000972,1029


In [ ]:
bankclient_embed5 = transactions.groupby('user_id')['transaction_dttm'].apply(lambda x: np.max(x) - np.min(x)).dt.days.astype('int16').to_frame() # важно
bankclient_embed5.columns = [f'bnk-max_date_diff' for i in bankclient_embed5.columns]
bankclient_embed5.head()

,bnk-max_date_diff
user_id,
1,361
2,372
3,363
4,370
5,364


In [ ]:
main_bankclient_embed = bankclient_embed2.merge(bankclient_embed4, how='left', left_on='user_id', right_index=True
                                        ).merge(bankclient_embed5, how='left', left_on='user_id', right_index=True)

del bankclient_embed2, bankclient_embed4, bankclient_embed5
gc.collect()

dtype_bankclient = list()
for x in main_bankclient_embed.dtypes.tolist():
    if x == 'int64' or x == 'int32' or x == 'int16':
        dtype_bankclient.append('int16')
    elif x == 'float64' or x == 'float32':
        dtype_bankclient.append('float32')
    else:
        dtype_bankclient.append('object')
    
dtype_bankclient = dict(zip(main_bankclient_embed.columns.tolist(),dtype_bankclient))
main_bankclient_embed = main_bankclient_embed.astype(dtype_bankclient)
main_bankclient_embed.head()

,sum--1,sum-48,sum-50,sum-60,mean--1,mean-48,mean-50,mean-60,count--1,count-48,...,bnk-count-weekday-6-nhour-16,bnk-count-weekday-6-nhour-17,bnk-count-weekday-6-nhour-18,bnk-count-weekday-6-nhour-19,bnk-count-weekday-6-nhour-20,bnk-count-weekday-6-nhour-21,bnk-count-weekday-6-nhour-22,bnk-count-weekday-6-nhour-23,summs,bnk-max_date_diff
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,-1.834270e+05,0.0,0.0,0.000000,-217.073395,0.0,0.0,0.0,845.0,...,0.000000,0.000000,0.000000,0.008284,0.009467,0.007101,0.004734,0.004734,845,361
2,0.750996,-7.176788e+05,0.0,0.0,0.750996,-960.748108,0.0,0.0,1.0,747.0,...,0.012032,0.001337,0.001337,0.001337,0.000000,0.000000,0.000000,0.000000,748,372
3,0.000000,8.016948e+05,0.0,0.0,0.000000,827.342407,0.0,0.0,0.0,969.0,...,0.000000,0.001032,0.000000,0.000000,0.000000,0.000000,0.000000,0.001032,969,363
4,-0.378836,1.261829e+06,0.0,0.0,-0.378836,1747.685791,0.0,0.0,1.0,722.0,...,0.002766,0.000000,0.001383,0.006916,0.044260,0.049793,0.009682,0.000000,723,370
5,0.837259,-4.172432e+05,0.0,0.0,0.837259,-405.878601,0.0,0.0,1.0,1028.0,...,0.000972,0.001944,0.005831,0.001944,0.002915,0.000972,0.000000,0.000972,1029,364


In [ ]:
main_bankclient_embed.head()

,sum--1,sum-48,sum-50,sum-60,mean--1,mean-48,mean-50,mean-60,count--1,count-48,...,bnk-count-weekday-6-nhour-16,bnk-count-weekday-6-nhour-17,bnk-count-weekday-6-nhour-18,bnk-count-weekday-6-nhour-19,bnk-count-weekday-6-nhour-20,bnk-count-weekday-6-nhour-21,bnk-count-weekday-6-nhour-22,bnk-count-weekday-6-nhour-23,summs,bnk-max_date_diff
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,-1.834270e+05,0.0,0.0,0.000000,-217.073395,0.0,0.0,0.0,845.0,...,0.000000,0.000000,0.000000,0.008284,0.009467,0.007101,0.004734,0.004734,845,361
2,0.750996,-7.176788e+05,0.0,0.0,0.750996,-960.748108,0.0,0.0,1.0,747.0,...,0.012032,0.001337,0.001337,0.001337,0.000000,0.000000,0.000000,0.000000,748,372
3,0.000000,8.016948e+05,0.0,0.0,0.000000,827.342407,0.0,0.0,0.0,969.0,...,0.000000,0.001032,0.000000,0.000000,0.000000,0.000000,0.000000,0.001032,969,363
4,-0.378836,1.261829e+06,0.0,0.0,-0.378836,1747.685791,0.0,0.0,1.0,722.0,...,0.002766,0.000000,0.001383,0.006916,0.044260,0.049793,0.009682,0.000000,723,370
5,0.837259,-4.172432e+05,0.0,0.0,0.837259,-405.878601,0.0,0.0,1.0,1028.0,...,0.000972,0.001944,0.005831,0.001944,0.002915,0.000972,0.000000,0.000972,1029,364


2. Clickstream

In [ ]:
clickstream['date'] = clickstream['timestamp'].dt.date.astype('category')
clickstream['hour'] = clickstream['timestamp'].dt.hour.astype('category')
clickstream['weekday'] = clickstream['timestamp'].dt.dayofweek.astype('category')

In [ ]:
clickstream_embed2 = clickstream.pivot_table(index = 'user_id', 
                            values=['timestamp'],
                            columns=['date'],
                            aggfunc=['count']).fillna(0)

clickstream_embed2.head()

count                                                         \
         timestamp                                                          
date    2021-01-17 2021-01-18 2021-01-19 2021-01-20 2021-01-21 2021-01-22   
user_id                                                                     
1                0          0          0          0          0          0   
2                0          0          0          0          0          1   
3                0          0          0          0          0          0   
4                0          0          0          0          0          0   
5                0          0          2          0          0          1   

                                                     ...             \
                                                     ...              
date    2021-01-23 2021-01-24 2021-01-25 2021-01-26  ... 2021-08-05   
user_id                                              ...              
1                0          0          0          0  ...          0   
2                1          1          1          0  ...          0   
3                0          0          0          1  ...          1   
4                0          0          0          0  ...          1   
5                1          0          2          4  ...          1   

                                                                           \
                                                                            
date    2021-08-06 2021-08-07 2021-08-08 2021-08-09 2021-08-10 2021-08-11   
user_id                                                                     
1                0          0          0          0          0          0   
2                0          0          0          0          0          0   
3                0          0          0          0          1          0   
4                0          0          0          0          0          0   
5                0          1          2          2          0          1   

                                          
                                          
date    2021-08-12 2021-08-13 2021-08-14  
user_id                                   
1                0          0          0  
2                0          0          0  
3                0          0          0  
4                0          0          0  
5                0          0          0  

[5 rows x 210 columns]

In [ ]:
clickstream_embed3 = clickstream.pivot_table(index ='user_id', 
                            values=['new_uid'],
                            aggfunc=['nunique']).fillna(0)
clickstream_embed3.head()

,nunique
,new_uid
user_id,
1,4
2,6
3,6
4,11
5,6


In [ ]:
clickstream_embed4 = clickstream.groupby('user_id')['timestamp'].apply(lambda x: np.max(x) - np.min(x)).dt.days.astype('int16').to_frame()
clickstream_embed4.head()

,timestamp
user_id,
1,170
2,193
3,195
4,187
5,204


In [ ]:
clickstream_embed5 = clickstream.pivot_table(index = 'user_id', 
                            values=['timestamp'],
                            columns=['weekday','hour'],
                            aggfunc=['count']).fillna(0)                        
clickstream_embed5.head()                            

count                                             ...            \
        timestamp                                             ...             
weekday         0                                             ...    6        
hour            0   1   2    3    4    5    6    7    8    9  ...   14   15   
user_id                                                       ...             
1               0   0   0    0    1    0    1    1    3    3  ...  252  145   
2               3   4  10   10   14   13   12   18   19   20  ...   77   54   
3               5   2   9   10    4   10   21   37   61   51  ...   38   55   
4              16   8  35  100  140  130  102  241  234  245  ...  106   64   
5               7  29  57   70   70   61   70   60   50   27  ...   63   65   

                                              
                                              
weekday                                       
hour      16   17   18   19   20  21  22  23  
user_id                                       
1        164  225  282  244  148  65   9   0  
2         62   66   86   84   57  32  14   6  
3         48   48   71   77   67  54  24  10  
4         33    8    3    4    9  12  11  18  
5         26   14    5    1    0   0   0   0  

[5 rows x 168 columns]

In [ ]:
clickstream_embed2.columns = [f'rtk-{str(i[0])}-{str(i[2])}' for i in clickstream_embed2.columns]
clickstream_embed3.columns = [f'rtk-{str(i[0])}-{str(i[1])}' for i in clickstream_embed3.columns]
clickstream_embed4.columns = [f'rtk-max_date_diff' for i in clickstream_embed4.columns] # важно
clickstream_embed5.columns = [f'rtk-{str(i[0])}-weekday-{str(i[2])}-nhour-{str(i[3])}' for i in clickstream_embed5.columns] # важно

In [ ]:
main_clickstream_embed = clickstream_embed2.merge(clickstream_embed3, left_on='user_id', right_index=True).merge(
                                            clickstream_embed4, left_on='user_id', right_index=True).merge(
                                            clickstream_embed5, left_on='user_id', right_index=True)

In [ ]:
# main_clickstream_embed.loc[0] = np.empty(len(main_clickstream_embed.columns)) # добавляем user_id = 0

del clickstream_embed2, clickstream_embed3, clickstream_embed4, clickstream_embed5
gc.collect()

# оптимизация памяти из бэйзлайна (думаю можно на SparseDtype заменить)
dtype_clickstream = list()
for x in main_clickstream_embed.dtypes.tolist():
    if x == 'int64' or x == 'int32' or x == 'int16':
          dtype_clickstream.append('int16')
    elif x == 'float64' or x == 'float32':
          dtype_clickstream.append('float32')
    else:
          dtype_clickstream.append('object')

dtype_clickstream = dict(zip(main_clickstream_embed.columns.tolist(),dtype_clickstream))
main_clickstream_embed = main_clickstream_embed.astype(dtype_clickstream)

In [ ]:
main_clickstream_embed.head()

,rtk-count-2021-01-17,rtk-count-2021-01-18,rtk-count-2021-01-19,rtk-count-2021-01-20,rtk-count-2021-01-21,rtk-count-2021-01-22,rtk-count-2021-01-23,rtk-count-2021-01-24,rtk-count-2021-01-25,rtk-count-2021-01-26,...,rtk-count-weekday-6-nhour-14,rtk-count-weekday-6-nhour-15,rtk-count-weekday-6-nhour-16,rtk-count-weekday-6-nhour-17,rtk-count-weekday-6-nhour-18,rtk-count-weekday-6-nhour-19,rtk-count-weekday-6-nhour-20,rtk-count-weekday-6-nhour-21,rtk-count-weekday-6-nhour-22,rtk-count-weekday-6-nhour-23
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,252,145,164,225,282,244,148,65,9,0
2,0,0,0,0,0,1,1,1,1,0,...,77,54,62,66,86,84,57,32,14,6
3,0,0,0,0,0,0,0,0,0,1,...,38,55,48,48,71,77,67,54,24,10
4,0,0,0,0,0,0,0,0,0,0,...,106,64,33,8,3,4,9,12,11,18
5,0,0,2,0,0,1,1,0,2,4,...,63,65,26,14,5,1,0,0,0,0


### 3. Сделаем эмбеддинги как предлагается в базовом решении

In [ ]:
# CLICKSTREAM
clickstream_embed = clickstream.pivot_table(index = 'user_id', 
                            values=['timestamp'],
                            columns=['cat_id'],
                            aggfunc=['count']).fillna(0) # считаем количество дат посещений каждой категории сайтов
clickstream_embed.columns = [f'rtk_{str(i[0])}-{str(i[2])}' for i in clickstream_embed.columns]
clickstream_embed.loc[0] = np.empty(len(clickstream_embed.columns))
clickstream_embed.head() # Сколько посещений каждой категории сайта

,rtk_count-1,rtk_count-2,rtk_count-3,rtk_count-8,rtk_count-11,rtk_count-12,rtk_count-13,rtk_count-14,rtk_count-15,rtk_count-19,...,rtk_count-1819,rtk_count-1840,rtk_count-1848,rtk_count-1849,rtk_count-1850,rtk_count-1853,rtk_count-1856,rtk_count-1857,rtk_count-1858,rtk_count-1861
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,33.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,52.0,0.0,0.0,0.0,0.0,233.0,0.0,1.0,41.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,278.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,94.0,0.0,11.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# TRANSACTIONS
bankclient_embed = transactions.pivot_table(index = 'user_id', 
                            values=['transaction_amt'],
                            columns=['mcc_code'],
                            aggfunc=['sum','mean', 'count']).fillna(0)
bankclient_embed.columns = [f'bank_{str(i[0])}-{str(i[2])}' for i in bankclient_embed.columns]
bankclient_embed.head() # Сумма транзакций, средняя сумма тразакций по категории, количество транзакций в категории

,bank_sum--1,bank_sum-742,bank_sum-763,bank_sum-780,bank_sum-1520,bank_sum-1711,bank_sum-1731,bank_sum-1740,bank_sum-1750,bank_sum-1761,...,bank_count-8931,bank_count-8999,bank_count-9211,bank_count-9222,bank_count-9223,bank_count-9311,bank_count-9399,bank_count-9402,bank_count-9405,bank_count-9406
user_id,,,,,,,,,,,,,,,,,,,,,
1,3.075678e+03,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,1.795567e+03,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,2.329993e+06,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
4,1.648119e+06,-1455.3394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,0.0,0.0
5,1.007214e+05,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
bankclient_embed.fillna(0, inplace=True)
clickstream_embed.fillna(0, inplace=True)

print(bankclient_embed.memory_usage().sum()/1024/1024, 'Mb', clickstream_embed.memory_usage().sum()/1024/1024, 'Mb')
dtype = pd.SparseDtype(np.int32, fill_value=0)
bankclient_embed = bankclient_embed.astype(dtype)
clickstream_embed = clickstream_embed.astype(dtype)
print(bankclient_embed.memory_usage().sum()/1024/1024, 'Mb', clickstream_embed.memory_usage().sum()/1024/1024, 'Mb')

199.2473373413086 Mb 60.33685302734375 Mb
24.50006103515625 Mb 6.69207763671875 Mb


### 4. Обучение модели

#### Подготовка данных

In [ ]:
# Соберём обучающую выборку, где на 1 правильный мэтч будет приходиться 35 неправильных

k=35
cor_dict = train.set_index('bank')['rtk'].to_dict()

train_bank_ids = train['bank']  # [(train.rtk!=0)]
train_rtk_ids = train[train.bank.isin(train_bank_ids)]['rtk'].drop_duplicates()
df_train = pd.DataFrame(train_bank_ids, columns=['bank'])
df_train['rtk'] = df_train['bank'].apply(lambda x: [cor_dict[x]] + train_rtk_ids.sample(k, random_state=x).values.tolist() )

df_train = df_train.explode('rtk')

train['bank+rtk'] = train['bank'].astype('str')+'_'+train['rtk'].astype('str')
df_train['bank+rtk'] = df_train['bank'].astype('str')+'_'+df_train['rtk'].astype('str')
df_train['target'] = df_train['bank+rtk'].isin(train['bank+rtk']).astype('int')

df_train.drop_duplicates('bank+rtk', inplace=True)
df_train.reset_index(inplace=True, drop=True)

print(df_train.shape,  df_train['target'].mean())
print(df_train.bank.nunique(), df_train.rtk.nunique(), df_train['bank+rtk'].nunique())

(632879, 4) 0.027779401749781555
17581 14672 632879


In [ ]:
df_train.head()

,bank,rtk,bank+rtk,target
0,2091,17319,2091_17319,1
1,2091,16816,2091_16816,0
2,2091,11645,2091_11645,0
3,2091,7274,2091_7274,0
4,2091,14858,2091_14858,0


In [ ]:
X_train=df_train.merge(bankclient_embed, how='left', left_on='bank', right_index=True
                          ).merge(clickstream_embed, how='left', left_on='rtk', right_index=True
                                      ).merge(cl_sv, how='left', left_on='rtk', right_index=True
                                                       ).merge(tr_sv, how='left', left_on='bank', right_index=True
                                                       ).fillna(0)
X_train.head()

#### CatBoostRanker

In [ ]:
cat_features = list(X_train.select_dtypes(["category"])) 

train_pool = Pool(data=X_train.drop(['bank', 'rtk', 'bank+rtk', 'target'], axis=1),
                  label = X_train['target'],
                  cat_features=cat_features,
                  group_id = X_train['bank'])

In [ ]:
def train_cb(train_pool):
    default_parameters = {
        'custom_metric': ["Recall", "MRR","AUC:hints=skip_train~false", "Precision", 'AverageGain:top=10'],
        'random_seed': 42,
        "loss_function": "YetiRank",
        "train_dir": "YetiRank",
        "metric_period": 500,
        "bootstrap_type": "Bayesian",
        "bagging_temperature": 0.66
    }

    parameters = {
        'iterations': 9000,
        'task_type': "CPU",
        **default_parameters
    }

    ranker = CatBoostRanker(**parameters)
    ranker.fit(train_pool, verbose=500, plot=False)
    return ranker

In [ ]:
print("Fitting model...")
model = train_cb(train_pool) 

Fitting model...
0:	total: 300ms	remaining: 45m
500:	total: 1m 16s	remaining: 21m 33s
1000:	total: 2m 30s	remaining: 20m 4s
1500:	total: 3m 44s	remaining: 18m 43s
2000:	total: 4m 59s	remaining: 17m 27s
2500:	total: 6m 15s	remaining: 16m 14s
3000:	total: 7m 29s	remaining: 14m 58s
3500:	total: 8m 42s	remaining: 13m 40s
4000:	total: 9m 56s	remaining: 12m 24s
4500:	total: 11m 9s	remaining: 11m 9s
5000:	total: 12m 24s	remaining: 9m 54s
5500:	total: 13m 39s	remaining: 8m 40s
6000:	total: 14m 53s	remaining: 7m 26s
6500:	total: 16m 7s	remaining: 6m 11s
7000:	total: 17m 22s	remaining: 4m 57s
7500:	total: 18m 37s	remaining: 3m 43s
8000:	total: 19m 52s	remaining: 2m 28s
8500:	total: 21m 6s	remaining: 1m 14s
8999:	total: 22m 21s	remaining: 0us


In [ ]:
model.save_model('sol_2504.cbm')

Feature Importance

In [ ]:
fi = model.get_feature_importance(data=train_pool, prettified=True, type='LossFunctionChange')
fi 

,Feature Id,Importances
0,rtk_count-535,3.643486e-03
1,rtk_count-273,2.892863e-03
2,rtk_count-289,2.858471e-03
3,rtk_count-165,2.428087e-03
4,rtk_count-251,2.414179e-03
...,...,...
1605,rtk_count-186,-1.997509e-08
1606,rtk_count-427,-1.569993e-07
1607,bank_mean-5735,-3.689183e-05
1608,bank_sum-5735,-4.291595e-05


#### Puzzle inference

In [ ]:
puzzle_bank_ids = puzzle['bank'].drop_duplicates()
puzzle_rtk_ids = puzzle['rtk'].drop_duplicates()
df_puzzle = pd.DataFrame(puzzle_bank_ids, columns=['bank'])
df_puzzle['rtk'] = df_puzzle['bank'].apply(lambda x: puzzle_rtk_ids.values.tolist())

df_puzzle = df_puzzle.explode('rtk')
df_puzzle.reset_index(inplace=True, drop=True)

In [ ]:
print(df_puzzle.shape, len(puzzle_bank_ids), len(puzzle_rtk_ids))

(24522304, 2) 4952 4952


In [ ]:
batch = 100
clf = CatBoostRanker()
clf.load_model('sol_2504.cbm')
for i in tqdm_notebook(range(len(puzzle_bank_ids)//batch+1)):
    bank_batch = puzzle_bank_ids[i*batch:(i+1)*batch]
    X_puzzle=df_puzzle[df_puzzle.bank.isin(bank_batch)].merge(bankclient_embed, how='left', left_on='bank', right_index=True
                                                        ).merge(clickstream_embed, how='left', left_on='rtk', right_index=True
                                                        ).merge(cl_sv, how='left', left_on='rtk', right_index=True
                                                       ).merge(tr_sv, how='left', left_on='bank', right_index=True
                                                       ).fillna(0)
    df_puzzle.loc[df_puzzle.bank.isin(bank_batch), 'predicts'] = clf.predict(X_puzzle[clf.feature_names_])
    del X_puzzle
    
df_puzzle['pred_rank'] = df_puzzle.groupby('bank')['predicts'].rank(ascending=False)

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
sub = df_puzzle[df_puzzle.pred_rank<=100].copy(deep=True)
sub.sort_values(by=['bank', 'predicts'], ascending=False, inplace=True)
sub['rtk'] = all_dicts['rtk_le'].inverse_transform(sub['rtk'].astype(int)-1)
sub['bank'] = all_dicts['bank_le'].inverse_transform(sub['bank']-1)

sub = sub.pivot_table(index='bank', values='rtk', aggfunc=list)
sub['rtk'] = sub['rtk'].apply(lambda x: str(x))
sub = sub.replace("'", '', regex=True)
sub.rename({'rtk':'rtk_list'}, axis=1).to_csv('puzzle_solution.csv', index=True)